In [5]:
import numpy as np

from sklearn.datasets import load_iris
ds = load_iris()

from sklearn.preprocessing import StandardScaler
for i in range(ds['data'].shape[1]):
    ds['data'][:, i] = StandardScaler().fit_transform(ds['data'][:, i].reshape(-1, 1)).flatten()

In [6]:
import pandas as pd

print(ds.data.shape)
print(ds.target_names)
pd.Series(ds.target).value_counts()

(150, 4)
['setosa' 'versicolor' 'virginica']


2    50
1    50
0    50
dtype: int64

In [8]:
from collections import defaultdict

In [15]:
class Variable:
    
    
    def __init__(self, name, value, derivative=None, is_number=False):
        self.name = name
        self.value = value
        if derivative is None:
            self.derivative = defaultdict(float)
            if not is_number:
                self.derivative[self.name] = 1
        else:
            self.derivative = derivative   
        self.is_number = is_number
        self.tolerance = 1e-8

        
    def __add__(self, other):
        derivative = defaultdict(float)
        leafs = list(set(self.derivative.keys()).union(other.derivative.keys()))
        for v in leafs:
            derivative[v] = 1*self.derivative[v] + 1*other.derivative[v]
        return Variable(
            '(%s + %s)' % (self.name, other.name),
            self.value + other.value,
            derivative,
            is_number=self.is_number and other.is_number
        )
        
        
    def __mul__(self, other):
        derivative = defaultdict(float)
        leafs = list(set(self.derivative.keys()).union(other.derivative.keys()))
        for v in leafs:
            derivative[v] = other.value*self.derivative[v] + self.value*other.derivative[v]
        return Variable(
            '(%s * %s)' % (self.name, other.name),
            self.value * other.value,
            derivative,
            is_number=self.is_number and other.is_number
        )
    
    
    def __neg__(self):
        derivative = defaultdict(float)
        for k, v in self.derivative.items():
            # write correct code
            derivative[k] = 0 # <<<--- REPLACE  IT
        return Variable(
            self.name[1:] if self.name.startswith('-') else '-%s' % self.name,
            -self.value,
            derivative,
            is_number=self.is_number
        )
    
    
    def __sub__(self, other):
        # write correct code
        return 0 # <<<--- REPLACE  IT
    
    
    def log(self):
        derivative = defaultdict(float)
        for k, v in self.derivative.items():
            # write correct code
            derivative[k] = 0 # <<<--- REPLACE  IT
        return Variable(
            'log(%s)' % self.name,
            np.log(self.value),
            derivative,
            is_number=self.is_number
        )
    
    
    def exp(self):
        derivative = defaultdict(float)
        for k, v in self.derivative.items():
            # write correct code
            derivative[k] = 0 # <<<--- REPLACE  IT
        return Variable(
            'exp(%s)' % self.name,
            np.exp(self.value),
            derivative,
            is_number=self.is_number
        )
    
    
    def __pow__(self, other):
        if not other.is_number:
            raise NotImplementedError()
        derivative = defaultdict(float)
        for k, v in self.derivative.items():
            derivative[k] = other.value*(self.value**(other.value - 1))*v
        return Variable(
            '(%s^(%s))' % (self.name, other.name),
            self.value**other.value,
            derivative,
            is_number=self.is_number
        )
    
    
    def __truediv__(self, other):
        # write correct code
        return 0 # <<<--- REPLACE  IT
    

    def __repr__(self):
        return 'Variable(\n  %s,\n  %0.2f,\n  %s\n)' % (
            self.name, 
            self.value,
            '' if self.derivative is None else '; '.join(
                [('d[%s]/d[%s]=%0.2f' % (self.name, k, v)) 
                 for (k, v) in self.derivative.items()]
            )
        )

In [16]:
var_w0 = Variable('w0', 0.3)
var_w1 = Variable('w1', 0.4)

var_x0 = Variable('x0', 1, is_number=True)
var_x1 = Variable('x1', 5, is_number=True)

var_z = var_w0*var_x0 + var_w1*var_x1

var_z

Variable(
  ((w0 * x0) + (w1 * x1)),
  2.30,
  d[((w0 * x0) + (w1 * x1))]/d[w0]=1.00; d[((w0 * x0) + (w1 * x1))]/d[w1]=5.00
)

In [17]:
var_1 = Variable('1', 1, is_number=True)

var_p = var_1 / (var_1 + (-var_z).exp())

var_p

0

In [18]:
var_y = Variable('1', 1, is_number=True)

var_loss = -var_y*var_p.log() - (var_1 - var_y)*(var_1 - var_p).log()

var_loss

AttributeError: 'int' object has no attribute 'log'

In [19]:
var_loss.derivative['w0'], var_loss.derivative['w1']

(-0.09709677932757939, -0.006666838987567296)

In [20]:
var_w0 = Variable('w0', 0)
var_w1 = Variable('w1', 0)
var_w2 = Variable('w2', 0)
var_w3 = Variable('w3', 0)
var_w4 = Variable('w4', 0)

learning_rate = 0.001
n_epochs = 50
var_1 = Variable('1', 1, is_number=True)

for _ in range(n_epochs):
    acc_epoch = 0
    loss_epoch = 0
    for i in range(ds['data'].shape[0]):
        x = ds['data'][i, :]
        y = int(ds['target'][i] != 0)

        var_y = Variable(str(y), y, is_number=True)

        var_z = \
            var_w0 + \
            var_w1*Variable('x1', x[0], is_number=True) + \
            var_w2*Variable('x2', x[1], is_number=True) + \
            var_w3*Variable('x3', x[2], is_number=True) + \
            var_w4*Variable('x4', x[3], is_number=True)

        var_p = var_1/(var_1 + (-var_z).exp())
            
        y_pred = int(var_p.value >= 0.5)
        if y_pred == y:
            acc_epoch += 1

        if y == 0:
            var_loss = -(var_1 - var_p).log()
        else:
            var_loss = -var_p.log()
        loss_epoch += var_loss.value                

        var_w0.value -= learning_rate*var_loss.derivative[var_w0.name]
        var_w1.value -= learning_rate*var_loss.derivative[var_w1.name]
        var_w2.value -= learning_rate*var_loss.derivative[var_w2.name]
        var_w3.value -= learning_rate*var_loss.derivative[var_w3.name]
        var_w4.value -= learning_rate*var_loss.derivative[var_w4.name]

    acc_epoch /= ds['data'].shape[0]
    loss_epoch /= ds['data'].shape[0]
    print(acc_epoch, loss_epoch)

AttributeError: 'int' object has no attribute 'value'